In [48]:
### API Mini Project - Logan Larson

This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [14]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = '*' # removed before posting to GitHub

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [19]:
# First, import the relevant modules

import pandas as pd
import requests

In [16]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?" + "&start_date=2019-01-01&end_date=2019-01-02&api_key=" + API_KEY
req = requests.get(url)

In [17]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

req.text

'{"dataset_data":{"limit":null,"transform":null,"column_index":null,"column_names":["Date","Open","High","Low","Close","Change","Traded Volume","Turnover","Last Price of the Day","Daily Traded Units","Daily Turnover"],"start_date":"2019-01-01","end_date":"2019-01-02","frequency":"daily","data":[["2019-01-02",null,70.85,68.25,69.85,null,null,null,null,null,null]],"collapse":null,"order":null}}'

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [35]:
# Task 1

url2 = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?" + "&start_date=2017-01-01&end_date=2017-12-31&api_key=" + API_KEY
req2 = requests.get(url2)

In [36]:
# Task 2

data = req2.json()
cols = data["dataset_data"]["column_names"] # extract column names from nested dictionary
rows = data["dataset_data"]["data"] # extract row names from nested dictionary
df = pd.DataFrame(rows, columns=cols) # transform into pandas dataframe
df.head(3)

,Date,Open,High,Low,Close,Change,Traded Volume,Turnover,Last Price of the Day,Daily Traded Units,Daily Turnover
0,2017-12-29,51.76,51.94,51.45,51.76,NaN,34640.0,1792304.0,None,None,None
1,2017-12-28,51.65,51.82,51.43,51.60,NaN,40660.0,2099024.0,None,None,None
2,2017-12-27,51.45,51.89,50.76,51.82,NaN,57452.0,2957018.0,None,None,None


In [45]:
# Task 3

open_max = 0.0 # dummy variable that will update when a new max is found, starting from zero
open_min = float("inf") # dummy variable that will update when a new min is found, starting from infinity
for row in data["dataset_data"]["data"]:
    if row[1] != None:
       if row[1] > open_max:
          open_max = row[1]
       if row[1] < open_min:
          open_min = row[1]
print("The highest opening price in 2017 was ${0:.2f} while the lowest was ${1:.2f}." .format(round(open_max, 2), round(open_min, 2)))

The highest opening price in 2017 was $53.11 while the lowest was $34.00.


In [47]:
# Task 4

delta = 0.0 # dummy variable to track the change in price

for row in data["dataset_data"]["data"]: # this function will calculate the change in price
    if row[1] != None:
       if row[2]-row[3] > delta:
          delta = row[2]-row[3]
print("The largest change was ${0:.2f}." .format(round(delta, 2)))

The largest change was $2.81.


In [63]:
# Task 5

delta2 = {}
close_early =  data["dataset_data"]["data"][0][4]

for row in data["dataset_data"]["data"][1:]:
    close = row[4]
    delta2[row[0]] = abs(close_early - close)
    close_early = close

delta_date, delta_value = max(delta.items(), key=lambda k:k[1])

print("The largest difference between any two days was ${0:.2f}.".format(round(delta_value, 2), (delta_date)))

The largest difference between any two days was $2.56.


In [67]:
# Task 6

volume = [row[6] for row in data["dataset_data"]["data"]]
average = sum(volume) / len(volume)
print("The average daily trading volume was ${0:.2f}." .format(round(average, 2)))

The average daily trading volume was $89124.34.
